# Part 1: Cleaning the data: First steps
<i style="font-size: 0.94em">This notebook is part of a series detailing the creation of models for predicting NFL game outcomes. For the table of contents listing all notebooks in the series, <a href="0.Introduction_NFL_Prediction.ipynb">click here.</a></i>

To predict games, we need data to train on. To train, we must turn the raw data into features. To get the features, the data must be clean. So let's get to it!

The main dataset used here is <a href="https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016?select=NFL+Play+by+Play+2009-2018+%28v5%29.csv">this publically available table of play data from 2009-2018.</a> It contains a row for every play in every game from 2009-2018.
    
Our first step will be to load the file in. Specifically, we'll read it in from a CSV to a Pandas dataframe.

<b style="color: red;">Note: This requires about 2.3 gigabytes of memory. If you are running on a shared server (IE, in Binder), this may cause problems.</b> The code is presented here mainly for reference; you can <a href="2.NFL_Game_Data_Extraction.ipynb">skip to section two</a> or go back <a href="0.Introduction_NFL_Prediction.ipynb">to the table of contents</a> if you need to. The code in the other sections will run without issue.

In [1]:
import pandas as pd
import sys

data_directory = "./Data/"
file_name = "Original_Giant_CSV2009-2018.csv"
df = pd.read_csv(data_directory+file_name)
print("DF read successfully")
print(f"Size of raw dataframe is {sys.getsizeof(df):,d} bytes.")

/tmp/ipykernel_16621/1086634220.py:6: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_directory+file_name)


DF read successfully
Size of raw dataframe is 2,312,710,920 bytes.


We've read in the data successfully, but 2.3 gigabytes seems bigger than it needs to be. It could cause problems on older machines or shared servers. Is there anything we can do about this? 

Let's inspect the data. Normally, we might use df.describe() here. However, this is a large file and it's not running on a top-of-the-line machine, so we'll just stick with df.head(). I'll also check the size of the dataframe while here.

In [2]:
df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


We have over 255 columns! We can remove a few of the columns that are clearly not needed now in order to speed up processing.

Let's look at the columns and then remove a few that we don't need. The above hides some of the column headers, and printing out the columns directly will also omit most of the column names. However, we can see every column conveniently by setting Pandas' default number of max columns to 255 and then calling df.head again:

In [3]:
pd.set_option("display.max_columns", None)
df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackl

Getting rid of a few unnecessary columns will make the data easier to understand and use, as well as making the program run a bit faster. We can follow a few simple principles to perform this basic feature choice:
<ol>
    <li>This model focuses on numeric and categorical data. Text columns, such as the description of the play, may prove useful in a model that can take them into account, but that is beyond the scope of the current study. Hence, text columns that cannot easily be converted to categorical data are dropped.</li>
    <li>We won't analyze individual players, so we can drop columns involving individual players.</li>
    <li>My model does not use "second-order" statistics calculated from other, more immediately measurable statistics, such as EPA(estimated points added). Why not include these data points that are central in many football experts' analyses? Football analysis has been around for a long time, but many metrics such as quarterback rating were not designed with modern data science best-practices in mind. Many of these second-order statistics involve simplifying assumptions that may or may not be accurate. A proper machine-learning model may do better without them. </li>
    <li>Events like multiple laterals and on-side kicks are tracked in the data set, but are sufficiently rare in the modern NFL as to have minimal use in a predictive model. That is, the additional dimensions would slow down the code but are unlikely to provide additional accuracy due to their sparsity. If anything, including the additional dimensions about rare events like on-side kicks, multiple laterals, or extra point conversions returned by the defense would contribute to overfitting.</li>
</ol>

<p>In view of the above considerations, we can reduce the number of columns from 255 to a much-more-manageable 115, using a list of column titles and the df.drop() method to do so.</p>

In [4]:
columns_to_drop = ["desc", "no_score_prob", "opp_fg_prob", "opp_safety_prob", "opp_td_prob", 
                   "fg_prob", "safety_prob", "td_prob", "extra_point_prob", "two_point_conversion_prob", 
                   "ep", "epa", "total_home_epa", "total_away_epa", "total_home_rush_epa", 
                   "total_away_rush_epa", "total_home_pass_epa", "total_away_pass_epa", "air_epa", 
                   "yac_epa", "comp_air_epa", "comp_yac_epa", "total_home_comp_air_epa", 
                   "total_away_comp_air_epa", "total_home_comp_yac_epa", "total_away_comp_yac_epa", 
                   "total_home_raw_air_epa", "total_away_raw_air_epa", "total_home_raw_yac_epa", 
                   "total_away_raw_yac_epa", "wp", "def_wp", "home_wp", "away_wp", "wpa", "home_wp_post", 
                   "away_wp_post", "total_home_rush_wpa", "total_away_rush_wpa", "total_home_pass_wpa", 
                   "total_away_pass_wpa", "air_wpa", "yac_wpa", "comp_air_wpa", "comp_yac_wpa", 
                   "total_home_comp_air_wpa", "total_away_comp_air_wpa", "total_home_comp_yac_wpa", 
                   "total_away_comp_yac_wpa", "total_home_raw_air_wpa", "total_away_raw_air_wpa", 
                   "total_home_raw_yac_wpa", "total_away_raw_yac_wpa", "lateral_reception", "lateral_rush", 
                   "lateral_return", "lateral_recovery", "passer_player_id", "passer_player_name", 
                   "receiver_player_id", "receiver_player_name", "rusher_player_id", "rusher_player_name", 
                   "lateral_receiver_player_id", "lateral_receiver_player_name", "lateral_rusher_player_id", 
                   "lateral_rusher_player_name", "lateral_sack_player_id", "lateral_sack_player_name", 
                   "interception_player_id", "interception_player_name", "lateral_interception_player_id", 
                   "lateral_interception_player_name", "punt_returner_player_id", "punt_returner_player_name", 
                   "lateral_punt_returner_player_id", "lateral_punt_returner_player_name", 
                   "kickoff_returner_player_name", "kickoff_returner_player_id", 
                   "lateral_kickoff_returner_player_id", "lateral_kickoff_returner_player_name", 
                   "punter_player_id", "punter_player_name", "kicker_player_name", "kicker_player_id", 
                   "own_kickoff_recovery_player_id", "own_kickoff_recovery_player_name", "blocked_player_id", 
                   "blocked_player_name", "tackle_for_loss_1_player_id", "tackle_for_loss_1_player_name", 
                   "tackle_for_loss_2_player_id", "tackle_for_loss_2_player_name", "qb_hit_1_player_id", 
                   "qb_hit_1_player_name", "qb_hit_2_player_id", "qb_hit_2_player_name", 
                   "forced_fumble_player_1_team", "forced_fumble_player_1_player_id", 
                   "forced_fumble_player_1_player_name", "forced_fumble_player_2_team", 
                   "forced_fumble_player_2_player_id", "forced_fumble_player_2_player_name", 
                   "solo_tackle_1_team", "solo_tackle_2_team", "solo_tackle_1_player_id", 
                   "solo_tackle_2_player_id", "solo_tackle_1_player_name", "solo_tackle_2_player_name", 
                   "assist_tackle_1_player_id", "assist_tackle_1_player_name", "assist_tackle_1_team", 
                   "assist_tackle_2_player_id", "assist_tackle_2_player_name", "assist_tackle_2_team", 
                   "assist_tackle_3_player_id", "assist_tackle_3_player_name", "assist_tackle_3_team", 
                   "assist_tackle_4_player_id", "assist_tackle_4_player_name", "assist_tackle_4_team", 
                   "pass_defense_1_player_id", "pass_defense_1_player_name", "pass_defense_2_player_id", 
                   "pass_defense_2_player_name", "fumbled_1_team", "fumbled_1_player_id", 
                   "fumbled_1_player_name", "fumbled_2_player_id", "fumbled_2_player_name", 
                   "fumbled_2_team", "fumble_recovery_1_team", "fumble_recovery_1_yards", 
                   "fumble_recovery_1_player_id", "fumble_recovery_1_player_name", "fumble_recovery_2_team", 
                   "fumble_recovery_2_yards", "fumble_recovery_2_player_id", "fumble_recovery_2_player_name",
                   "own_kickoff_recovery_td", "own_kickoff_recovery", "defensive_two_point_attempt",
                   "defensive_extra_point_attempt", "defensive_extra_point_conv", "defensive_two_point_conv", 
                   "penalty_player_id", "penalty_player_name"]

df = df.drop(columns_to_drop, axis=1)
print(f"New number of columns: {len(df.columns)}")
print(f"New size of df: {sys.getsizeof(df):,d} bytes")

New number of columns: 108
New size of df: 812,695,846 bytes


<h2>That's great! We've cut the size down to about 1/3 of the original! </h2>

Now let's think about the datatypes in our NFL dataset. Because we're reading in as a CSV, the datatypes are not saved in the file itself, and Pandas has already given us a warning about columns with mixed types. We can look at the types and make sure they match up with what we expect.

<h1>Dealing with team names</h1>
The datatypes above seem reasonable. However, we have a few too many 'objects'. String data in Pandas gets stored as an 'object'. One object takes up far more space than an int or a simpler type, and for such a large table, it adds up. Now, for some pieces of data, such as the name of a player or a description of a play, we may have no choice but to use strings/objects. Since this isn't an NLP model, we already dropped all those. The objects that remain are mostly NFL team names. How can we save space here?

Since there are only 32 teams in the NFL, we can assign each team a number and store that number. Since these numbers would only go from 1 to 32 (or 0 to 31), we could put them in a small integer and save a large amount of space over the entire table. Pandas makes this easy by having a categorical data type.

Before setting up the categories, we'll need to fix something else. Several NFL teams changed names and locations from 2009 to 2018. For example, the San Diego Chargers moved to Los Angeles, but remained the same team. So we'll need to update the old names in order to have 32 teams and track data for each team across multiple seasons.

You can see in the code below that the number of unique values in the "home_team" column, for example, is 35. This is obviously not right.

In [5]:
dif_teams = len(df["home_team"].unique())
print(f"Number of different teams in NFL: {dif_teams} ")

def update_old_team_names(df_to_replace: pd.DataFrame):
    df_to_replace.replace(to_replace="STL", value="LA", inplace=True)
    df_to_replace.replace(to_replace="SD", value="LAC", inplace=True)
    df_to_replace.replace(to_replace="JAC", value="JAX", inplace=True)

update_old_team_names(df)
dif_teams = len(df["home_team"].unique())
print(f"Number of different teams in NFL: {dif_teams} ")

Number of different teams in NFL: 35 
Number of different teams in NFL: 32 


Now we can convert the columns with team abbreviations to categorical data. Each team is mapped to an integer. We want those mappings to match across all the columns to ensure consistency. So we convert the first column to categorical, then extract its CategoricalDType (an object of a special Pandas datatype specifying the mappings) and pass this as a parameter to the subsequent conversions to categorical dtype.

In [6]:
df["home_team"] = df["home_team"].astype("category")
list_of_categories = df["home_team"].cat.categories
teams_categorical = pd.api.types.CategoricalDtype(categories=list_of_categories)

# posteam_type is 'home' or 'away' and thus should not use the same categories as below, so it's not on the list
columns_to_change = ["home_team", 
                    "away_team",
                    "posteam",
                    "defteam",
                    "side_of_field",
                    "timeout_team",
                    "td_team",
                    "return_team",
                    "penalty_team"]
df[columns_to_change] = df[columns_to_change].astype(teams_categorical)
print(f"Size of df after converting teams to categorical: {sys.getsizeof(df):,d} bytes")

Size of df after converting teams to categorical: 623,045,198 bytes


<h2>Great, we've reduced the memory usage of the df even more. </h2>
Now let's see if we can convert some of the other object/string columns into categories. We'll have a look at df.head() again, passing in 30 as a parameter so that we can see enough rows to make a smart choice. After viewing this, we can convert the other columns to categorical. Here, we don't need consistency across columns, so we skip the step of making a CategoricalDtype object.

In [7]:
df.head(10)

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,return_team,return_yards,penalty_team,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,900.0,1800.0,3600.0,Half1,0,1,0,1,NaN,0.0,15:00,TEN 30,0,0,kickoff,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,PIT,39.0,NaN,NaN,0,NaN,NaN
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,893.0,1793.0,3593.0,Half1,0,1,0,1,1.0,0.0,14:53,PIT 42,10,5,pass,5.0,0,0,1.0,0,0,0,short,left,-3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,NaN,NaN,0,NaN,NaN
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,856.0,1756.0,3556.0,Half1,0,1,0,1,2.0,0.0,14:16,PIT 47,5,2,run,-3.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,right,end,NaN,NaN,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0,NaN,NaN
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,815.0,1715.0,3515.0,Half1,0,1,0,1,3.0,0.0,13:35,PIT 44,8,2,pass,0.0,1,0,1.0,0,0,0,deep,right,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0,NaN,NaN
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,807.0,1707.0,3507.0,Half1,0,1,0,1,4.0,0.0,13:27,PIT 44,8,2,punt,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0,NaN,NaN
5,162,2009091000,PIT,TEN,TEN,away,PIT,TEN,98.0,2009-09-10,796.0,1696.0,3496.0,Half1,0,2,0,1,1.0,0.0,13:16,TEN 2,10,0,run,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,middle,NaN,NaN,NaN,NaN,NaN,3,3,0.0,NaN,NaN,3.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
more_columns_to_change = ['posteam_type', 'game_half', 'play_type', 'pass_length', 'pass_location', 
                     'run_location', 'run_gap', 'field_goal_result', 'extra_point_result', 'two_point_conv_result', 
                     'replay_or_challenge_result', 'penalty_type']
df[more_columns_to_change] = df[more_columns_to_change].astype("category")
print(f"Size of df after converting more columns categorical dtype: {sys.getsizeof(df):,d} bytes")

Size of df after converting more columns categorical dtype: 397,693,773 bytes


<h2>A few other considerations before finishing up...</h2>
Do we really need both the yardline_100 and yrdln columns? Football announcers will normally say things like "On the Dallas 25-yard line," and this information gets recorded in the yrdln column. Do we really need both the team name and a yardline, or is it more useful to represent this as a number from 0 to 100? For mathematical analysis, the latter is clearly better. Hence, we drop the yrdln column and keep only yardline_100.

Similar considerations lead to dropping the 'time' column as well.

Finally, we can convert the 'game_date', which is now an object, to be a special date-time type that should use less space and be easier to work with.

In [9]:
df = df.drop(["time", "yrdln"], axis=1)
df["game_date"] = pd.to_datetime(df["game_date"])
print(f"Final size of cleaned df: {sys.getsizeof(df):,d} bytes")


Final size of cleaned df: 315,046,738 bytes


<h2>Wow, our cleaned dataframe is less than one-seventh the size of the original!</h2>

We could get further reductions by converting some of the int and float columns to boolean, as these columns contain either 0 or 1. To truly optimize, we could change some of the int64 and float64 to smaller integers and floats, as an NFL game does not require the massive numbers or granular precision of these types.

However, our dataframe is small already enough to run quickly on a cheap laptop or budget web service, and we might have to change the smaller types back into larger ones for input into the neural network model later. We'll go ahead and export the df in order to more quickly forge ahead with feature extraction and analysis!

<h1>Exporting data</h1>
We'll use the .pickle format to save space and maintain datatypes. Once we've exported this cleaned data, we're ready for the next step in our data pipeline: Turning the play data into game data and season data!

<h2>Check out the next step in this fun NFL model <a href="./2.NFL_Game_Data_Extraction.ipynb">by clicking right here!!</a></h2>

In [10]:
output_file_name = "cleaned_data_pickle.pkl"
df.to_pickle(data_directory+output_file_name)